# Converter for the Financial Filings at the Sec

In [43]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('importFiles/2011q4.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t")
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        
dirname = "2011q4"

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

19621233
10897460
288540
838134
24168


In [44]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+-------------------------------------------------------+--------------+---------+----------+--------+-------+------------------+------------+
|    | adsh                 | tag                                                   | version      |   coreg |    ddate |   qtrs | uom   |            value |   footnote |
|----+----------------------+-------------------------------------------------------+--------------+---------+----------+--------+-------+------------------+------------|
|  0 | 0000808450-11-000104 | AcceleratedShareRepurchaseProgramAdjustment           | us-gaap/2011 |     nan | 20111031 |      4 | USD   |     -2e+07       |        nan |
|  1 | 0001193125-11-299700 | AcceleratedShareRepurchaseProgramAdjustment           | us-gaap/2011 |     nan | 20101231 |      0 | USD   |      6e+06       |        nan |
|  2 | 0001047469-11-008939 | AcceleratedShareRepurchaseProgramAdjustment           | us-gaap/2011 |     nan | 20100930 |      3 | USD   |     -1

In [45]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+----------+--------+--------+---------+---------+-------------------------------------------------------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                                   | version      | plabel                                                                                                                                                                                       |   negating |
|----+----------------------+----------+--------+--------+---------+---------+-------------------------------------------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------

In [46]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+---------+---------------------------------------------+-------+-------------+----------+----------------------+------------+----------------------------------------+---------------------------------+---------------------+-------------+----------+----------------------+------------+----------------------------------------+---------------------------------+--------------+-----------+-------------+--------------------------------------+---------------+-------+--------+-------+--------+----------+------+------+----------+-----------------------+-----------+----------+----------------------------+---------+-----------------------------------------+
|    | adsh                 |     cik | name                                        |   sic | countryba   | stprba   | cityba               | zipba      | bas1                                   | bas2                            | baph                | countryma   | stprma   | cityma               | zipma      | mas1 

In [47]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))

+----+-----------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+-------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | tag                                     

In [48]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+


In [49]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                               0000808450-11-000104
tag         AcceleratedShareRepurchaseProgramAdjustment
version                                    us-gaap/2011
coreg                                               NaN
ddate                                          20111031
qtrs                                                  4
uom                                                 USD
value                                       -20000000.0
footnote                                            NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [50]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

# Create a single SymbolFinancialsDto

In [ ]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    sfDto.symbol = val["symbol"].to_string(index = False).upper()
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False)
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(str(result))
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 1126 Dtos size: 504 Bs: 248 Cf: 256 Ic: 0 Json size: 119kb.
file 1 of 8016 exportFiles/2011q4/0000950123-11-088081.json stored in 23507.041ms.
FilterDfNum size: 115 Dtos size: 65 Bs: 38 Cf: 27 Ic: 0 Json size: 14kb.
file 2 of 8016 exportFiles/2011q4/0001019687-11-003123.json stored in 2514.834ms.
FilterDfNum size: 217 Dtos size: 105 Bs: 48 Cf: 57 Ic: 0 Json size: 25kb.
file 3 of 8016 exportFiles/2011q4/0001019687-11-003144.json stored in 4706.648ms.
FilterDfNum size: 207 Dtos size: 127 Bs: 60 Cf: 67 Ic: 0 Json size: 29kb.
file 4 of 8016 exportFiles/2011q4/0001176256-11-000754.json stored in 4227.711ms.
FilterDfNum size: 68 Dtos size: 38 Bs: 24 Cf: 14 Ic: 0 Json size: 9kb.
file 5 of 8016 exportFiles/2011q4/0001477932-11-002068.json stored in 1499.378ms.
FilterDfNum size: 237 Dtos size: 104 Bs: 66 Cf: 38 Ic: 0 Json size: 26kb.
file 6 of 8016 exportFiles/2011q4/0000082473-11-000018.json stored in 4848.657ms.
FilterDfNum size: 155 Dtos size: 114 Bs: 86 Cf: 28 Ic: 0 Json s

FilterDfNum size: 102 Dtos size: 70 Bs: 50 Cf: 20 Ic: 0 Json size: 16kb.
file 54 of 8016 exportFiles/2011q4/0001354488-11-003706.json stored in 2331.835ms.
FilterDfNum size: 83 Dtos size: 58 Bs: 40 Cf: 18 Ic: 0 Json size: 13kb.
file 55 of 8016 exportFiles/2011q4/0001372167-11-000039.json stored in 1915.429ms.
FilterDfNum size: 286 Dtos size: 134 Bs: 67 Cf: 67 Ic: 0 Json size: 32kb.
file 56 of 8016 exportFiles/2011q4/0001387131-11-002196.json stored in 6238.773ms.
FilterDfNum size: 181 Dtos size: 111 Bs: 57 Cf: 54 Ic: 0 Json size: 26kb.
file 57 of 8016 exportFiles/2011q4/0001437749-11-007437.json stored in 4110.118ms.
FilterDfNum size: 149 Dtos size: 80 Bs: 48 Cf: 32 Ic: 0 Json size: 17kb.
file 58 of 8016 exportFiles/2011q4/0001477932-11-002100.json stored in 3402.257ms.
FilterDfNum size: 66 Dtos size: 64 Bs: 64 Cf: 0 Ic: 0 Json size: 15kb.
file 59 of 8016 exportFiles/2011q4/0001501919-11-000028.json stored in 1604.246ms.
FilterDfNum size: 362 Dtos size: 199 Bs: 94 Cf: 105 Ic: 0 Json si

FilterDfNum size: 258 Dtos size: 136 Bs: 72 Cf: 64 Ic: 0 Json size: 33kb.
file 107 of 8016 exportFiles/2011q4/0001140361-11-048995.json stored in 6166.162ms.
FilterDfNum size: 158 Dtos size: 101 Bs: 62 Cf: 39 Ic: 0 Json size: 29kb.
file 108 of 8016 exportFiles/2011q4/0001144204-11-057051.json stored in 3798.179ms.
FilterDfNum size: 292 Dtos size: 121 Bs: 34 Cf: 87 Ic: 0 Json size: 35kb.
file 109 of 8016 exportFiles/2011q4/0001144204-11-057094.json stored in 7048.948ms.
FilterDfNum size: 264 Dtos size: 145 Bs: 77 Cf: 68 Ic: 0 Json size: 35kb.
file 110 of 8016 exportFiles/2011q4/0001144204-11-057151.json stored in 6174.186ms.
FilterDfNum size: 186 Dtos size: 149 Bs: 105 Cf: 44 Ic: 0 Json size: 33kb.
file 111 of 8016 exportFiles/2011q4/0001144204-11-057159.json stored in 4042.873ms.
FilterDfNum size: 181 Dtos size: 106 Bs: 56 Cf: 50 Ic: 0 Json size: 24kb.
file 112 of 8016 exportFiles/2011q4/0001144204-11-057309.json stored in 4020.856ms.
FilterDfNum size: 161 Dtos size: 100 Bs: 62 Cf: 38 

FilterDfNum size: 164 Dtos size: 90 Bs: 50 Cf: 40 Ic: 0 Json size: 19kb.
file 160 of 8016 exportFiles/2011q4/0001010549-11-001110.json stored in 3968.601ms.
FilterDfNum size: 117 Dtos size: 69 Bs: 42 Cf: 27 Ic: 0 Json size: 15kb.
file 161 of 8016 exportFiles/2011q4/0001013762-11-002690.json stored in 2777.866ms.
FilterDfNum size: 133 Dtos size: 83 Bs: 39 Cf: 44 Ic: 0 Json size: 19kb.
file 162 of 8016 exportFiles/2011q4/0001014897-11-000212.json stored in 3206.596ms.
FilterDfNum size: 175 Dtos size: 99 Bs: 62 Cf: 37 Ic: 0 Json size: 22kb.
file 163 of 8016 exportFiles/2011q4/0001019687-11-003204.json stored in 4168.03ms.
FilterDfNum size: 144 Dtos size: 72 Bs: 42 Cf: 30 Ic: 0 Json size: 16kb.
file 164 of 8016 exportFiles/2011q4/0001024020-11-000063.json stored in 3470.793ms.
FilterDfNum size: 136 Dtos size: 82 Bs: 43 Cf: 39 Ic: 0 Json size: 19kb.
file 165 of 8016 exportFiles/2011q4/0001052918-11-000544.json stored in 3279.722ms.
FilterDfNum size: 114 Dtos size: 73 Bs: 35 Cf: 38 Ic: 0 Jso

FilterDfNum size: 169 Dtos size: 94 Bs: 58 Cf: 36 Ic: 0 Json size: 22kb.
file 213 of 8016 exportFiles/2011q4/0001171520-11-000753.json stored in 4119.937ms.
FilterDfNum size: 142 Dtos size: 106 Bs: 68 Cf: 38 Ic: 0 Json size: 25kb.
file 214 of 8016 exportFiles/2011q4/0001193125-11-271324.json stored in 3424.346ms.
FilterDfNum size: 273 Dtos size: 122 Bs: 56 Cf: 66 Ic: 0 Json size: 29kb.
file 215 of 8016 exportFiles/2011q4/0001193125-11-271377.json stored in 6441.008ms.
FilterDfNum size: 160 Dtos size: 89 Bs: 50 Cf: 39 Ic: 0 Json size: 22kb.
file 216 of 8016 exportFiles/2011q4/0001193125-11-271396.json stored in 3834.035ms.
FilterDfNum size: 270 Dtos size: 180 Bs: 83 Cf: 97 Ic: 0 Json size: 43kb.
file 217 of 8016 exportFiles/2011q4/0001193125-11-271810.json stored in 6371.126ms.
FilterDfNum size: 802 Dtos size: 193 Bs: 82 Cf: 111 Ic: 0 Json size: 44kb.
file 218 of 8016 exportFiles/2011q4/0001193125-11-271844.json stored in 18548.668ms.
FilterDfNum size: 156 Dtos size: 125 Bs: 66 Cf: 59 I

FilterDfNum size: 236 Dtos size: 103 Bs: 46 Cf: 57 Ic: 0 Json size: 25kb.
file 266 of 8016 exportFiles/2011q4/0001204459-11-002769.json stored in 5096.352ms.
FilterDfNum size: 77 Dtos size: 42 Bs: 24 Cf: 18 Ic: 0 Json size: 9kb.
file 267 of 8016 exportFiles/2011q4/0001255294-11-000605.json stored in 1777.539ms.
FilterDfNum size: 107 Dtos size: 53 Bs: 36 Cf: 17 Ic: 0 Json size: 11kb.
file 268 of 8016 exportFiles/2011q4/0001255294-11-000607.json stored in 2407.017ms.
FilterDfNum size: 69 Dtos size: 42 Bs: 24 Cf: 18 Ic: 0 Json size: 9kb.
file 269 of 8016 exportFiles/2011q4/0001255294-11-000609.json stored in 1585.146ms.
FilterDfNum size: 151 Dtos size: 104 Bs: 58 Cf: 46 Ic: 0 Json size: 22kb.
file 270 of 8016 exportFiles/2011q4/0001255294-11-000613.json stored in 3306.119ms.
FilterDfNum size: 156 Dtos size: 90 Bs: 39 Cf: 51 Ic: 0 Json size: 20kb.
file 271 of 8016 exportFiles/2011q4/0001255294-11-000616.json stored in 3439.81ms.
FilterDfNum size: 79 Dtos size: 48 Bs: 31 Cf: 17 Ic: 0 Json s

FilterDfNum size: 71 Dtos size: 31 Bs: 12 Cf: 19 Ic: 0 Json size: 7kb.
file 319 of 8016 exportFiles/2011q4/0001411879-11-000040.json stored in 1771.224ms.
FilterDfNum size: 306 Dtos size: 130 Bs: 64 Cf: 66 Ic: 0 Json size: 29kb.
file 320 of 8016 exportFiles/2011q4/0001445305-11-003015.json stored in 7343.524ms.
FilterDfNum size: 114 Dtos size: 58 Bs: 27 Cf: 31 Ic: 0 Json size: 14kb.
file 321 of 8016 exportFiles/2011q4/0001499785-11-000031.json stored in 2971.132ms.
FilterDfNum size: 318 Dtos size: 135 Bs: 76 Cf: 59 Ic: 0 Json size: 31kb.
file 322 of 8016 exportFiles/2011q4/0000029989-11-000007.json stored in 7603.67ms.
FilterDfNum size: 228 Dtos size: 118 Bs: 72 Cf: 46 Ic: 0 Json size: 29kb.
file 323 of 8016 exportFiles/2011q4/0000895419-11-000048.json stored in 5114.978ms.
FilterDfNum size: 278 Dtos size: 90 Bs: 52 Cf: 38 Ic: 0 Json size: 21kb.
file 324 of 8016 exportFiles/2011q4/0000950123-11-090775.json stored in 6828.877ms.
FilterDfNum size: 441 Dtos size: 186 Bs: 129 Cf: 57 Ic: 0 

FilterDfNum size: 228 Dtos size: 91 Bs: 48 Cf: 43 Ic: 0 Json size: 21kb.
file 371 of 8016 exportFiles/2011q4/0001193125-11-276717.json stored in 5415.688ms.
FilterDfNum size: 204 Dtos size: 38 Bs: 20 Cf: 18 Ic: 0 Json size: 8kb.
file 372 of 8016 exportFiles/2011q4/0001193125-11-276941.json stored in 4875.287ms.
FilterDfNum size: 413 Dtos size: 108 Bs: 52 Cf: 56 Ic: 0 Json size: 25kb.
file 373 of 8016 exportFiles/2011q4/0001193125-11-277040.json stored in 9392.26ms.
FilterDfNum size: 325 Dtos size: 142 Bs: 78 Cf: 64 Ic: 0 Json size: 33kb.
file 374 of 8016 exportFiles/2011q4/0001193125-11-277134.json stored in 7373.419ms.
FilterDfNum size: 238 Dtos size: 167 Bs: 98 Cf: 69 Ic: 0 Json size: 39kb.
file 375 of 8016 exportFiles/2011q4/0001193125-11-277136.json stored in 5411.233ms.
FilterDfNum size: 267 Dtos size: 134 Bs: 78 Cf: 56 Ic: 0 Json size: 34kb.
file 376 of 8016 exportFiles/2011q4/0001193125-11-277141.json stored in 6133.288ms.
FilterDfNum size: 171 Dtos size: 99 Bs: 59 Cf: 40 Ic: 0 

FilterDfNum size: 532 Dtos size: 185 Bs: 65 Cf: 120 Ic: 0 Json size: 44kb.
file 424 of 8016 exportFiles/2011q4/0000092380-11-000088.json stored in 12644.304ms.
FilterDfNum size: 747 Dtos size: 334 Bs: 252 Cf: 82 Ic: 0 Json size: 75kb.
file 425 of 8016 exportFiles/2011q4/0000096289-11-000038.json stored in 17001.266ms.
FilterDfNum size: 713 Dtos size: 159 Bs: 66 Cf: 93 Ic: 0 Json size: 37kb.
file 426 of 8016 exportFiles/2011q4/0000104207-11-000102.json stored in 17345.246ms.
FilterDfNum size: 793 Dtos size: 197 Bs: 57 Cf: 140 Ic: 0 Json size: 46kb.
file 427 of 8016 exportFiles/2011q4/0000723125-11-000189.json stored in 18408.712ms.
FilterDfNum size: 109 Dtos size: 74 Bs: 52 Cf: 22 Ic: 0 Json size: 16kb.
file 428 of 8016 exportFiles/2011q4/0000897101-11-001807.json stored in 2662.437ms.
FilterDfNum size: 264 Dtos size: 116 Bs: 52 Cf: 64 Ic: 0 Json size: 26kb.
file 429 of 8016 exportFiles/2011q4/0000912593-11-000045.json stored in 6136.9ms.
FilterDfNum size: 852 Dtos size: 442 Bs: 266 Cf:

FilterDfNum size: 199 Dtos size: 100 Bs: 66 Cf: 34 Ic: 0 Json size: 24kb.
file 476 of 8016 exportFiles/2011q4/0000950123-11-092064.json stored in 4380.305ms.
FilterDfNum size: 307 Dtos size: 134 Bs: 70 Cf: 64 Ic: 0 Json size: 47kb.
file 477 of 8016 exportFiles/2011q4/0000950123-11-092071.json stored in 6568.099ms.
FilterDfNum size: 268 Dtos size: 145 Bs: 70 Cf: 75 Ic: 0 Json size: 37kb.
file 478 of 8016 exportFiles/2011q4/0000950123-11-092081.json stored in 6040.804ms.
FilterDfNum size: 387 Dtos size: 235 Bs: 89 Cf: 146 Ic: 0 Json size: 54kb.
file 479 of 8016 exportFiles/2011q4/0001047469-11-008693.json stored in 8380.279ms.
FilterDfNum size: 361 Dtos size: 106 Bs: 46 Cf: 60 Ic: 0 Json size: 24kb.
file 480 of 8016 exportFiles/2011q4/0001104659-11-057859.json stored in 7795.803ms.
FilterDfNum size: 479 Dtos size: 155 Bs: 78 Cf: 77 Ic: 0 Json size: 38kb.
file 481 of 8016 exportFiles/2011q4/0001104659-11-057930.json stored in 10427.27ms.
FilterDfNum size: 248 Dtos size: 140 Bs: 70 Cf: 70 